In [ ]:
import numpy as np
import jupyter_rfb


class Drawingapp(jupyter_rfb.RemoteFrameBuffer):
    
    def __init__(self):
        super().__init__()
        self.css_height = "500px"
        self.pixel_ratio = 1
        self.array = np.zeros((1, 1, 4), np.uint8)
        self.pen_colors = [(1, 0.2, 0, 1), (0, 1, 0.2, 1), (0.2, 0, 1, 1)]
        self.pen_index = 0        
    
    def receive_event(self, event):
        event_type = event.get("event_type", None)
        if event_type == "resize":
            # Take pixel ratio into account, use 8x less pixels (in each dim)
            self.pixel_ratio = event["pixel_ratio"] / 8
            w = int(event["width"] * self.pixel_ratio)
            h = int(event["height"] * self.pixel_ratio)
            self.array = np.zeros((h, w, 4), np.uint8)
            self.send_frame(self.array)
        if event_type == "pointer_down":
            if event["button"] == 1:
                # Draw
                x = int(event["x"] * self.pixel_ratio)
                y = int(event["y"] * self.pixel_ratio)
                self.array[y, x] = 255 * np.array(self.pen_colors[self.pen_index])
                self.send_frame(self.array)
            elif event["button"] == 2:
                # Toggle color
                self.pen_index = (self.pen_index + 1) % len(self.pen_colors)


app = Drawingapp()
app